In [1]:
%load_ext autoreload
%autoreload 2

# Parsing function signatures out of js code

In [2]:
from jy.util import js_files

list(js_files)

['test01']

In [3]:
test01_js_code = js_files['test01']

In [4]:
from jy import add_js_funcs

js = add_js_funcs(test01_js_code)

In [5]:
# js has two methods called bar and foo
sorted([x for x in dir(js) if not x.startswith('_')])

['add_one', 'bar', 'foo', 'obj', 'prop', 'with_arrow_func', 'with_let']

In [6]:
# they mirror the signatures of the underlying JS functions
from dol.signatures import Sig
Sig(js.foo)

<Sig (a, b='hello', c=3)>

In [7]:
Sig(js.bar)

<Sig (green, eggs='food', and=True, ham=4)>

In [8]:
# Calling this function returns a string (the code to call the underlying JS function)
js.foo(1, 'hi', 5)

'foo(1, "hi", 5)'

In [9]:
# Notice that you can use positional or keyword arguments
# Also, notice that though "pop" is the name of js's attribute, 
# the function call string does indeed use the original full reference: 
# `func.assigned.to.nested.prop`
js.prop('up')

'func.assigned.to.nested.prop("up")'

In [10]:
# Notice that the python (signature) defaults are applied before translating to JS
js.bar(42)

'bar(42, "food", true, 4)'

In [11]:
# You can opt not to do this by specifying apply_defaults=False
# This will result in only injecting those inputs you specify in the js call string, 
# which will have the effect of letting JS apply it's defaults, what ever they are
alt_js = add_js_funcs(test01_js_code, apply_defaults=False)
alt_js.bar(42)

'bar(42)'

In [15]:
from jy.js_parse import *

js_code = 'const add_five = (x) => x + 5'
t = parse_js_code('let add_five = (x) => x + 5')
list(extract_func_name_and_params(t.body[0]))

[('add_five', [{'name': 'x'}])]

In [21]:
from itertools import islice

list(islice([1,2,3], None))

[1, 2, 3]

In [23]:
js_code = """
function foo(a, b="hello", c= 3) {
    return a + b.length * c
}

const bar = (y, z = 1) => y * z

// function assigned to a nested property
func.assigned.to.nested.prop = function (x) {
    return x + 3
}

"""

list(func_name_and_params_pairs(js_code))

[('foo',
  [{'name': 'a'},
   {'name': 'b', 'default': 'hello'},
   {'name': 'c', 'default': 3}]),
 ('bar', [{'name': 'y'}, {'name': 'z', 'default': 1}]),
 ('func.assigned.to.nested.prop', [{'name': 'x'}])]

# Misc tests

In [4]:
import jy
from functools import partial
from jy.bridge import _js_func_call, mk_py_binder_func

In [13]:
import i2

# _js_func_call('foo', 1, 'hi', 5)
ff = lambda a, b, c=True: a + b * c
params = i2.Sig(ff).params
f = mk_py_binder_func('foo', params)
f

functools.partial(<function _js_func_call at 0x1064204c0>, __sig=<Sig (a, b, c=True)>, __func_call_template='foo({inputs})', __value_trans=<function dflt_py_to_js_value_trans at 0x114514280>, __apply_defaults=True)

In [17]:
f(1, 'hio')

'foo(1, "hio", true)'